In [ ]:
!pip install torchcam

In [ ]:
from PIL import Image
#import splitfolders

import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

import torch
from torchvision.models import mobilenet_v3_large
from torch import nn, optim

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
bs = 64
crop_size = 224

test_transform = transforms.Compose([
    transforms.Resize(320),
    transforms.CenterCrop(crop_size),
    transforms.ToTensor(),
])


test_set = datasets.ImageFolder("/content/drive/MyDrive/belajar python/case study/forest fire/img/val", transform=test_transform)
testloader = DataLoader(test_set, batch_size=bs, shuffle=False)

In [ ]:
class ForestFireRecognition(nn.Module):
    def __init__(self):
        super().__init__()
        self.mnet = model = mobilenet_v3_large()
        self.mnet.classifier = nn.Sequential(
            nn.Hardswish(),
            nn.Linear(960,1280),
            nn.Hardswish(),
            nn.Dropout(0.2),
            nn.Linear(1280,2),
            nn.LogSoftmax(1)
        )
    def forward(self,x) :
        x = self.mnet(x)
        return x

    def features(self):
        return self.mnet.features

In [ ]:
model = ForestFireRecognition().to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/belajar python/case study/forest fire/model/mobilenetv3_98.pth",map_location=torch.device('cpu')))
model.to(device)
target_layers = [model.features()]

In [ ]:
itr = iter(testloader)
feature,target = next(itr)
feature , target = feature.to(device), target.to(device)

In [ ]:
from torchvision.io.image import read_image
from torchvision.transforms.functional import normalize, resize, to_pil_image
from torchcam.methods import SmoothGradCAMpp,XGradCAM,ScoreCAM,GradCAMpp,LayerCAM

cam_extractor = XGradCAM(model,target_layers)

list_result = []

for i in range(1,feature.shape[0]):
    # Preprocess your data and feed it to the model
    out = model(feature[i].unsqueeze(0))
    # Retrieve the CAM by passing the class index and the model output
    activation_map = cam_extractor(out.squeeze(0).argmax().item(), out)
    result = overlay_mask(to_pil_image(feature[i]), to_pil_image(activation_map[0].squeeze(0), mode='F'), alpha=0.8)
    list_result.append(result)

In [ ]:
fig,axes = plt.subplots(5,5,figsize = (24,24))
for image, grad, ax in zip(feature,list_result,axes.flatten()):
    ax.imshow(grad)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
fig,axes = plt.subplots(5,5,figsize = (24,24))
for image, grad, ax in zip(feature,list_result,axes.flatten()):
    ax.imshow(grad)

Output hidden; open in https://colab.research.google.com to view.